In [2]:
import json
import csv
from collections import defaultdict
import hashlib

def create_primary_key(third_party_data_id, vertical):
    combined = f"{third_party_data_id}|{vertical}"
    return hashlib.md5(combined.encode()).hexdigest()

def to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def transform_jsonl_to_csv_with_verticals(input_file, output_file):
    data = defaultdict(dict)
    non_performance_keys = set()

    with open(input_file, 'r') as f:
        for line in f:
            item = json.loads(line)
            third_party_data_id = item['ThirdPartyDataId']
            
            has_performance_data = False
            
            # Store non-performance data
            for key, value in item.items():
                if not any(key.endswith(suffix) for suffix in ['_ctr', '_cpa', '_cpc']):
                    data[third_party_data_id][key] = value
                    non_performance_keys.add(key)
            
            # Store performance data
            for key, value in item.items():
                if key.endswith(('_ctr', '_cpa', '_cpc')):
                    has_performance_data = True
                    vertical, metric = key.rsplit('_', 1)
                    if vertical == 'overall':
                        vertical = 'Overall'
                    data[third_party_data_id].setdefault(vertical, {})[metric] = to_float(value)
            
            # If no performance data, add a null entry
            if not has_performance_data:
                data[third_party_data_id]['null_vertical'] = {'ctr': None, 'cpa': None, 'cpc': None}

    # Prepare CSV headers
    headers = ['PrimaryKey'] + sorted(list(non_performance_keys)) + ['Vertical', 'CTR', 'CPA', 'CPC']

    # Write CSV
    with open(output_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        for third_party_data_id, item_data in data.items():
            base_row = {k: item_data.get(k, '') for k in non_performance_keys}
            verticals = [v for v in item_data.keys() if v not in non_performance_keys]
            
            if not verticals:
                verticals = ['null_vertical']
            
            for vertical in verticals:
                row = base_row.copy()
                vertical_value = None if vertical == 'null_vertical' else vertical
                row['PrimaryKey'] = create_primary_key(third_party_data_id, vertical_value)
                row['Vertical'] = vertical_value
                metrics = item_data.get(vertical, {})
                row['CTR'] = metrics.get('ctr', None)
                row['CPA'] = metrics.get('cpa', None)
                row['CPC'] = metrics.get('cpc', None)
                writer.writerow(row)


# Usage
transform_jsonl_to_csv_with_verticals('/Users/adamhunter/Documents/3rd_party_element_pipeline/data/jsonl/pinecone_data.jsonl', '/Users/adamhunter/Documents/3rd_party_element_pipeline/data/jsonl/pinecone_data.csv')

In [2]:
import pandas as pd

def create_sample_csv(input_csv_file, output_csv_file):
    # Load the original CSV file
    df = pd.read_csv(input_csv_file)
    
    # Calculate the number of rows for the sample (1/4 of the total rows)
    sample_size = len(df) // 4
    
    # Create a sample DataFrame
    sample_df = df.head(sample_size)
    
    # Write the sample DataFrame to a new CSV file
    sample_df.to_csv(output_csv_file, index=False)

# Usage
create_sample_csv('/Users/adamhunter/Documents/3rd_party_element_pipeline/data/csv/element_performance.csv', '/Users/adamhunter/Documents/3rd_party_element_pipeline/data/jsonl/pinecone_data_sample.csv')
